# DIY AD

Minimalistic implementation of AD in python.
The implementation is supposed to be minimal both in terms of lines-of-code and the concepts required for its implementation.
Namely, we will use exactly one core concept: linearizations.

In [ ]:
import numpy as np
from collections import defaultdict
import jax

A linearization can be though of as the implicit Jacobian matrix of a function (yes, the name "linearization" is misleading as it might imply having an offset that it does not have here).
We need our implicit Jacobian matrix to have four attributes: a forward pass (Jacobian-Vector-Product), a backwards pass (Vector-Jacobian-Product), a reference to the parameters with respect to which we are going to differentiate to, and the value of the linearized function.
The last attribute is required in order to amend the Jacobian.

The forward pass of the Jacobian reapplies all linearized operators of our function in the order in which they were originally called.
A simple recursion into the individual Jacobians of the operators is sufficient if each of them applies the chain rule correctly.
The backwards pass is slightly trickier as we need to watch out for accumulating gradients.
For this purpose of tracking accumulating gradients, our backwards pass will share a global `Tape` to which the gradient for each input to a function is accumulated on.

In [ ]:
def identiy(*t, tape): return t


class Linearization():
    def __init__(self, p, fwd=identiy, bwd=identiy, _wrt=None):
        self.p = p
        self._fwd, self._bwd = fwd, bwd
        # Reference initial parameters w.r.t. which we want to differentiate
        self._wrt = self if _wrt is None else _wrt

    def __call__(self, *t, tape=None):
        if tape is not None:
            return self._fwd(*t, tape=tape)
        # Outermost call of Linearization (i.e. without a tape yet)
        # `tape` serves as shared dict for gradient accumulation
        tape = defaultdict(lambda: 0.)
        o = self._fwd(*t, tape=tape)
        assert len(o) == 1 if len(tape) == 0 else True
        return o[0] if len(tape) == 0 else (tape[self._wrt], )

    @property
    def T(self):
        return self.__class__(None, self._bwd, self._fwd, _wrt=self._wrt)

    def __repr__(self):
        return f"{self.__class__.__name__}({self.p}, {self._fwd}, {self._bwd})"

Let's start by defining two simple functions (and their linearizations): `exp` and `add`.
First, let's have a look at the unary function `exp`.
Its Jacobian is a diagonal matrix as it is a point-wise operator.
In terms of code, the forward and reverse of the linearization are thus simple multiplications.
The diagonal operator, i.e. the multiplication, is amended once to the left of the Jacobian for the forward pass and once to the right for the backwards pass.

Notice how in the implementation below the computation of the diagonal is hoisted out of the forward and reverse mode using a closure.
This specific choice of rematerialization strategy can be easily configured by writing custom rules for functions.
A custom rule is nothing more than a simple `isinstance` check for a `Linearization` in a function.

The binary operator `add` is slightly more elaborate in that it takes two arguments and has to handle all permutations of linearizations versus no linearizations as input.
The `isinstance` checks thus have four cases but conceptually works the same as `exp`.

In [ ]:
def exp(pl):
    if isinstance(pl, Linearization):
        y = exp(pl.p)
        def fwd(*t, tape):
            t = pl(*t, tape=tape)
            assert len(t) == 1
            return (y * t[0], )
        def bwd(*t, tape):
            assert len(t) == 1
            t = pl.T(y * t[0], tape=tape)
            assert len(t) == 1
            tape[pl] += t[0]
            return t
        return Linearization(y, fwd, bwd, _wrt=pl._wrt)
    return np.exp(pl)


def add(pl_l, pl_r):
    if isinstance(pl_l, Linearization) and not isinstance(pl_r, Linearization):
        def bwd(*t, tape):
            t = pl_l.T(*t, tape=tape)
            assert len(t) == 1
            tape[pl_l] += t[0]
            return t
        return Linearization(pl_l.p + pl_r, pl_l, bwd, _wrt=pl_l._wrt)
    elif not isinstance(pl_l, Linearization) and isinstance(pl_r, Linearization):
        def bwd(*t, tape):
            t = pl_r.T(*t, tape=tape)
            assert len(t) == 1
            tape[pl_r] += t[0]
            return t
        return Linearization(pl_l + pl_r.p, pl_r, bwd, _wrt=pl_r._wrt)
    elif isinstance(pl_l, Linearization) and isinstance(pl_r, Linearization):
        def fwd(*t, tape):
            tn = pl_l(*t, tape=tape)
            assert len(tn) == 1
            c = tn[0]
            tn = pl_r(*t, tape=tape)
            assert len(tn) == 1
            c += tn[0]
            return (c, )
        def bwd(*t, tape):
            assert len(t) == 1 and pl_l._wrt is pl_r._wrt
            tape[pl_l] += t[0]
            tape[pl_r] += t[0]
            return (pl_l.T(*t, tape=tape), pl_r.T(*t, tape=tape))
        return Linearization(pl_l.p + pl_r.p, fwd, bwd, _wrt=pl_l._wrt)
    else:
        return pl_l + pl_r


def cost(p):
    p = exp(p)
    return add(exp(p), p)


p0, t = 3.14, 2.
p0lin = Linearization(np.array(p0))
j = cost(p0lin)
j(t), j.T(t)

In [ ]:
jax.jvp(lambda p: jax.numpy.exp(jax.numpy.exp(p)) + jax.numpy.exp(p), (p0, ), (t, ))[1], jax.vjp(lambda p: jax.numpy.exp(jax.numpy.exp(p)) + jax.numpy.exp(p), p0)[1](t)

In [ ]:
p0 = 1e-2 * np.arange(0, 9)
ones = np.ones((9, ))
y = exp(p0)
j = exp(exp(Linearization(p0)))
j(p0), j.T(ones)

In [ ]:
_, jj_T = jax.vjp(lambda x: jax.numpy.exp(jax.numpy.exp(x)), p0.astype(float))
_, jj_at_p0 = jax.jvp(lambda x: jax.numpy.exp(jax.numpy.exp(x)), (p0.astype(float), ), (p0, ))
jj_at_p0, jj_T(ones)

How best to rematerialize in the Jacobian is a difficult question.
The approach taken here is a flexible albeit lazy one: We let the user do it explicitly when defining the Jacobian.
However, the design of building forward and backwards pass in general incentivizes sharing memory between both passes.
Furthermore, it is trivial to hoist out constants via closures.

In [ ]:
def weighted_reduction(pl, n=32, n_cols=3):
    if isinstance(pl, Linearization):
        def fwd(*t, tape):
            return (weighted_reduction(*pl(*t, tape=tape)), )
        def bwd(*t, tape):
            assert len(t) == 1
            p_shp = pl.p.reshape(3, -1).shape
            t_T, indices = np.zeros(p_shp), np.arange(n) % p_shp[0]
            for i, idx in enumerate(indices):
                super_expensive_weights = np.ones(p_shp[1:])
                t_T[idx] += t[0][i] * super_expensive_weights
            tape[pl] += t_T
            return pl.T(t_T.reshape(pl.p.shape), tape=tape)
        return Linearization(weighted_reduction(pl.p), fwd, bwd, _wrt=pl._wrt)
    p = pl.reshape(n_cols, -1)
    y, indices = np.zeros((n, )), np.arange(n) % p.shape[0]
    for i, idx in enumerate(indices):
        super_expensive_weights = np.ones(p.shape[1:])
        y[i] = np.sum(p[idx] * super_expensive_weights)
    return y


p0lin = Linearization(np.arange(12, dtype=float))
y2 = weighted_reduction(p0lin.p)
j = weighted_reduction(p0lin)
j(p0lin.p), j.T(y2, )

In [ ]:
def f(p): return weighted_reduction(exp(p))

p0 = 1e-2 * np.arange(0, 9)
f0 = f(p0)
f_ones = np.ones(f0.shape)

j = f(Linearization(p0))
j(p0), j.T(f_ones)

In [ ]:
def sum(pl):
    if isinstance(pl, Linearization):
        def fwd(*t, tape):
            return (sum(*pl(*t, tape=tape)), )
        def bwd(*t, tape):
            assert len(t) == 1
            tape[pl] = t = t[0] * np.ones(pl.p.shape)
            return pl.T(t, tape=tape)
        return Linearization(sum(pl.p), fwd, bwd, _wrt=pl._wrt)
    return np.sum(pl)

p0 = np.arange(0, 9, dtype=float)
j = sum(Linearization(p0))
j(p0), j.T(1.)

In [ ]:
def pow(pl, exponent):
    if isinstance(pl, Linearization) and not isinstance(exponent, Linearization):
        yl = exponent * pow(pl.p, exponent - 1)
        def fwd(*t, tape):
            t = pl(*t, tape=tape)
            assert len(t) == 1
            return (yl * t[0], )
        def bwd(*t, tape):
            assert len(t) == 1
            tape[pl] = t = yl * t[0]
            return pl.T(t, tape=tape)
        return Linearization(pow(pl.p, exponent), fwd, bwd, _wrt=pl._wrt)
    elif not isinstance(pl, Linearization) and not isinstance(exponent, Linearization):
        return pl**exponent
    else:
        raise NotImplementedError()


y = pow(3., 3)
j = pow(Linearization(3.), 3)
j.T(4.), jax.vjp(lambda x: x**3, 3.)[1](4.)

In [ ]:
data = 0.

def h(p):
    return add(sum(pow(add(-data, weighted_reduction(exp(p))), 2)), sum(pow(p, 2)))

j = h(Linearization(p0))
j(p0), j.T(1.)